# Data retrieval from localbitcoins API

In [1]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timezone

In [2]:
def make_request(url_base,url):
    r = requests.get(url_base+url)
    file_json = r.json()
    return file_json

def json_to_pandas(file_json:dict):
    return pd.DataFrame.from_dict(file_json)


### Calculation exchange rate of reference to further math on pct_change:
def exchange_reference(df, date_reference):
    
    memory = {}
        
    for currency_code in df["currency_code"].unique():
        
        if currency_code != "VED":
            date_reference = date_reference
        else:
            date_reference = "2021-10-10"

    
        ## Select currency for calculations:
        df_exchange = df[df['currency_code'] == currency_code ]

        ## Select date to calculate reference

        df_date_reference = df_exchange.loc[(df_exchange.date).dt.date == pd.Timestamp(date_reference)]

        ### Averare exchange rate for day of reference:

        implicit_exchange_reference = df_date_reference.implicit_exchange.mean()
        
        memory[currency_code] = implicit_exchange_reference

    ### Percentual Variation

#     equation = ((df_exchange.implicit_exchange / implicit_exchange_reference)-1)*100
    
    
    return memory

## Currencies

In [3]:
url_base = 'https://localbitcoins.com/'
# Currencies:
url = "api/currencies"
currency_code = make_request(url_base,url)
df_currency_code = json_to_pandas(currency_code['data'])
# df_currency_code.info()
# df_currency_code.head()

In [4]:
## Extract country information related to currency
currency_name = []
currency_is_altcoin = []
for index, row in df_currency_code.iterrows():
    currency_name.append(row[0]["name"])
    currency_is_altcoin.append(row[0]["altcoin"])
    
## Reorganize on df

df_currency_code['currencies'] = currency_name
df_currency_code['altcoin'] = currency_is_altcoin
df_currency_code = df_currency_code.reset_index()

df_currency_code.head()

,index,currencies,currency_count,altcoin
0,ADA,Cardano (ADA),185,True
1,AED,United Arab Emirates Dirham (AED),185,False
2,AFN,Afghan Afghani (AFN),185,False
3,ALL,Albanian Lek (ALL),185,False
4,AMD,Armenian Dram (AMD),185,False


### Average price of btc by country (in local currency)

In [5]:
url = "bitcoinaverage/ticker-all-currencies/"
price_btc = make_request(url_base,url)
time_stamp = datetime.now().timestamp()

In [6]:
df_price_btc = json_to_pandas(price_btc)

#transpose data frame. Index is now by Country:

df_price_btc = df_price_btc.T
df_price_btc['time_stamp'] = time_stamp
df_price_btc = df_price_btc.reset_index()
print(df_price_btc.info())
df_price_btc.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       79 non-null     object 
 1   volume_btc  79 non-null     object 
 2   rates       79 non-null     object 
 3   avg_1h      32 non-null     object 
 4   avg_6h      55 non-null     object 
 5   avg_12h     63 non-null     object 
 6   avg_24h     79 non-null     object 
 7   time_stamp  79 non-null     float64
dtypes: float64(1), object(7)
memory usage: 5.1+ KB
None


,index,volume_btc,rates,avg_1h,avg_6h,avg_12h,avg_24h,time_stamp
0,EUR,4.40429028,{'last': '36434.86'},37213.50,37778.65,38385.64,38035.50,1.641979e+09
1,SEK,1.12125766,{'last': '408952.79'},430820.74,422659.44,418378.43,424892.89,1.641979e+09
2,CRC,0.23214975,{'last': '27503884.92'},NaN,27478385.50,26528091.31,26632417.48,1.641979e+09
3,KWD,0.01323293,{'last': '13602.43'},NaN,NaN,NaN,13602.43,1.641979e+09
4,NGN,3.94512955,{'last': '24066001.18'},24194638.49,24147227.97,24188476.29,24000849.51,1.641979e+09


## Merge and clean dataframe

In [7]:
df_market = df_price_btc.merge(df_currency_code, how='right')

# Drop the rows with NAN values in time_stamp
df_market.dropna(subset=['time_stamp'], inplace=True)

# Add date from time_stamp
df_market['date'] = datetime.fromtimestamp(time_stamp, tz=timezone.utc)
df_market = df_market[['time_stamp','date','index','currencies','volume_btc','avg_24h','avg_12h','avg_6h','avg_1h','altcoin']]
df_market.rename(columns = {'index':'currency_code'}, inplace = True)
print(df_market.info())
df_market.head(10)


### I should include some data formatting so info is stored in the right format

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79 entries, 0 to 183
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   time_stamp     79 non-null     float64            
 1   date           79 non-null     datetime64[ns, UTC]
 2   currency_code  79 non-null     object             
 3   currencies     79 non-null     object             
 4   volume_btc     79 non-null     object             
 5   avg_24h        79 non-null     object             
 6   avg_12h        63 non-null     object             
 7   avg_6h         55 non-null     object             
 8   avg_1h         32 non-null     object             
 9   altcoin        79 non-null     bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(1), object(7)
memory usage: 6.2+ KB
None


,time_stamp,date,currency_code,currencies,volume_btc,avg_24h,avg_12h,avg_6h,avg_1h,altcoin
0,1.641979e+09,2022-01-12 09:21:58.985103+00:00,ADA,Cardano (ADA),0.01336275,37417.45,NaN,NaN,NaN,True
4,1.641979e+09,2022-01-12 09:21:58.985103+00:00,AMD,Armenian Dram (AMD),0.00072003,20832465.31,NaN,NaN,NaN,False
6,1.641979e+09,2022-01-12 09:21:58.985103+00:00,AOA,Angolan Kwanza (AOA),0.01129946,30001433.70,30001433.70,30001433.70,30080301.33,False
7,1.641979e+09,2022-01-12 09:21:58.985103+00:00,ARS,Argentine Peso (ARS),0.66683617,8531156.82,8688880.97,8892601.28,NaN,False
8,1.641979e+09,2022-01-12 09:21:58.985103+00:00,AUD,Australian Dollar (AUD),0.49623186,61590.97,62576.34,62462.12,NaN,False
15,1.641979e+09,2022-01-12 09:21:58.985103+00:00,BDT,Bangladeshi Taka (BDT),0.00092800,3526939.66,3559743.86,3559743.86,3559743.86,False
21,1.641979e+09,2022-01-12 09:21:58.985103+00:00,BOB,Bolivian Boliviano (BOB),0.11668888,310376.53,314167.86,NaN,NaN,False
22,1.641979e+09,2022-01-12 09:21:58.985103+00:00,BRL,Brazilian Real (BRL),0.46913198,244630.69,242475.88,255414.79,NaN,False
25,1.641979e+09,2022-01-12 09:21:58.985103+00:00,BWP,Botswanan Pula (BWP),0.00081410,577324.65,NaN,NaN,NaN,False
26,1.641979e+09,2022-01-12 09:21:58.985103+00:00,BYN,Belarusian Ruble (BYN),0.18547755,113939.45,114152.38,115091.88,114890.57,False


In [8]:
## Adding avg_24h exchange rate USD/BTC for inmediate comparison:

df_market["avg_24h_usd"] = df_market[df_market['currency_code']=="USD"].avg_24h.tolist()[0]

In [9]:
### Formatting data types:

# Float values:
for column_label in ['volume_btc','avg_24h','avg_12h','avg_6h','avg_1h','avg_24h_usd']:

    df_market[column_label] = pd.to_numeric(df_market[column_label])

df_market.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79 entries, 0 to 183
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   time_stamp     79 non-null     float64            
 1   date           79 non-null     datetime64[ns, UTC]
 2   currency_code  79 non-null     object             
 3   currencies     79 non-null     object             
 4   volume_btc     79 non-null     float64            
 5   avg_24h        79 non-null     float64            
 6   avg_12h        63 non-null     float64            
 7   avg_6h         55 non-null     float64            
 8   avg_1h         32 non-null     float64            
 9   altcoin        79 non-null     bool               
 10  avg_24h_usd    79 non-null     float64            
dtypes: bool(1), datetime64[ns, UTC](1), float64(7), object(2)
memory usage: 6.9+ KB


In [10]:
## Calculating implicit exchange rate:
df_market['implicit_exchange'] = df_market['avg_24h'].divide(df_market["avg_24h_usd"], fill_value = None)

In [11]:
# Drop the rows with NAN values in time_stamp
df_market.dropna(subset=['time_stamp'], inplace=True)

In [12]:
df_market.head(2)

,time_stamp,date,currency_code,currencies,volume_btc,avg_24h,avg_12h,avg_6h,avg_1h,altcoin,avg_24h_usd,implicit_exchange
0,1.641979e+09,2022-01-12 09:21:58.985103+00:00,ADA,Cardano (ADA),0.013363,37417.45,NaN,NaN,NaN,True,44332.29,0.844022
4,1.641979e+09,2022-01-12 09:21:58.985103+00:00,AMD,Armenian Dram (AMD),0.000720,20832465.31,NaN,NaN,NaN,False,44332.29,469.916291


## Save information in .csv and sql database

In [13]:
# Saving csv by timestamp:
df_market.to_csv(f'./data/csv/{time_stamp}.csv',index=False)
# Saving historical data by appending:
df_market.to_csv(f'./data/csv/historical_data.csv',index=False, mode = 'a')

In [14]:
# Append information on sqlite database

from sqlalchemy import Column, Integer, Float, String, DateTime,Boolean, ForeignKey
from sqlalchemy.orm import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///data/CurrenciesDataBase.db')
print(type(engine))
df_market.to_sql(name="currencies_vs_btc", con=engine, if_exists="append", index=False)

<class 'sqlalchemy.engine.base.Engine'>


#### Calculation of pct_change taking 2021-10-03 as date of reference

In [15]:
# path_to_currencies = "sqlite:///data/CurrenciesDataBase.db"
# df_market = pd.read_sql('currencies_vs_btc', path_to_currencies)

# ### The pct calculations are not properly saved on the dataset unless I upload one previously saved. Why?
# # The pipeline defined solve the issue by saving the calculated data on a new dataset but the calculations
# # are done in the full dataset everytime the code runs.
# # This practice is not scalable

# # The pct could be calculated on the web interface for a given date selected by the users but this implies 
# # more processing time

In [16]:
# df = df_market.groupby(by=["date","currency_code"]).mean()
# df.reset_index(inplace=True)

# ### Dataset with implicit exchange of reference by currency:
# date_reference = "2021-10-03"
# exchange = exchange_reference(df, date_reference)
# exchange = pd.DataFrame.from_dict(exchange, orient="index")

# df['pct'] = df.apply(lambda x : ((x['implicit_exchange']/exchange.loc[x["currency_code"]])-1)*100, axis = 1 )

# ### Drop Null values:

# df.dropna(subset=['pct'], inplace=True)

In [17]:
# df.info()

In [18]:
# ### Saving on DB Version 2

# engine = create_engine('sqlite:///data/CurrenciesDataBase_V2.db')
# print(type(engine))
# df.to_sql(name="currencies_vs_btc", con=engine, if_exists="append", index=False)

In [19]:
now = dt.datetime.now()
print("Last Execution: ")
print(str(now))
print('Completed')

Last Execution: 
2022-01-12 10:21:59.382440
Completed
